In [208]:
import json
import os
import pandas as pd
import numpy as np
import asyncio
import websockets
import plotly.express as px
from datetime import datetime as dt
import seaborn as sns


import_filepath = ""

def get_project_settings(importFilepath):
    if os.path.exists(importFilepath):
        f = open(importFilepath, "r")
        project_settings = json.load(f)
        f.close()
        return project_settings
    else:
        return ImportError

In [89]:
from binance.spot import Spot

def query_binance_status():
    status = Spot().system_status()
    if status['status'] == 0:
        return True
    else:
        raise ConnectionError
        
def query_account(api_key, secret_key):
    return Spot(api_key=api_key, api_secret=secret_key).account()

In [198]:
def get_candlestick_data(symbol, timeframe, qty):
    raw_data = Spot().klines(symbol=symbol, interval=timeframe, limit=qty)
    converted_data = []
    for candle in raw_data:
        converted_candle = {
            'time': candle[0],
            'open': float(candle[1]),
            'high': float(candle[2]),
            'low': float(candle[3]),
            'close': float(candle[4]),
            'volume': float(candle[5]),
            'close_time': candle[6],
            'quote_asset_volume': float(candle[7]),
            'number_of_trades': int(candle[8]),
            'taker_buy_base_asset_volume': float(candle[9]),
            'taker_buy_quote_asset_volume': float(candle[10])
        }
        converted_data.append(converted_candle)
    return converted_data

def get_and_transform_binance_data(symbol, timeframe, qty):
    raw_data = get_candlestick_data(symbol=symbol, timeframe=timeframe, qty=qty)
    df_data = pd.DataFrame(raw_data)
    df_data['time'] = pd.to_datetime(df_data['time'], unit='ms')
    df_data['close_time'] = pd.to_datetime(df_data['close_time'], unit='ms')
    df_data['RedOrGreen'] = np.where((df_data['open'] < df_data['close']), 'Green', 'Red')
    return df_data

def query_spot_asset_list(quote_asset_symbol):
    symbol_dictionary = Spot().ticker_price
    symbol_dataframe = pd.DataFrame(symbol_dictionary['symbols'])
    quote_symbol_dataframe = symbol_dataframe.loc[symbol_dataframe['quoteAsset'].isin(quote_asset_symbol) ]
    quote_symbol_dataframe = quote_symbol_dataframe.loc[quote_symbol_dataframe['status'] == "TRADING"]
    return quote_symbol_dataframe

def get_historical_data(client, period_length, ticker):
    print(f"Getting data for {ticker}")
    end_date = datetime.datetime.now()
    start_date = end_date - datetime.timedelta(days = period_length)
    candle = client.get_historical_klines(ticker, Client.KLINE_INTERVAL_30MINUTE, str(start_date), str(end_date))
    df = pd.DataFrame(candle, columns=['dateTime', 'open', 'high', 'low', 'close', 'volume', 'closeTime', 'quoteAssetVolume', 'numberOfTrades', 'takerBuyBaseVol', 'takerBuyQuoteVol', 'ignore'])
    df['dateTime'] = pd.to_datetime(df.dateTime, unit='ms')
    df['dateTime_str'] = df.dateTime.dt.strftime("%Y-%m-%d %H-%M-%S")
    #df.set_index('dateTime', inplace=True)
    df = df.drop(['closeTime', 'quoteAssetVolume', 'numberOfTrades', 'takerBuyBaseVol','takerBuyQuoteVol', 'ignore'], axis=1)
    df.open = df.open.astype(float)
    df.high = df.high.astype(float)
    df.low = df.low.astype(float)
    df.close = df.close.astype(float)
    df.volume = df.volume.astype(float)
    return df
    

async def candle_stick_data():
    url = "wss://stream.binance.com:9443/ws/"
    first_pair = 'cfxbusd@kline_30m'
    async with websockets.connect(url+first_pair) as sock:
        pairs = '{"method": "SUBSCRIBE", "params": ["btcbusd@kline_30m","ethbusd@kline_30m" ],  "id": 1}' #other pairs

        await sock.send(pairs)
        print(f"> {pairs}")
        while True:
            resp = await sock.recv()
            print(f"< {resp}")

def get_return_per_interval(x, cols):
    for c in cols:
        x[f"{c}_return"] = (x[c]-x[c].shift(1))/x[c].shift(1)
    return x

In [199]:
from binance.client import Client
import pandas as pd
import datetime, time

api_key = project_settings['BinanceKeys']['API_Key']
api_secret = project_settings['BinanceKeys']['Secret_Key']
client = Client(api_key, api_secret)
client.API_URL = 'https://api.binance.com/api'

tickers = ["CFXBUSD", "BTCBUSD"]
historical_days = 30
ticker_data =[]
for idx, ticker in enumerate(tickers):
    data = get_historical_data(client, historical_days, ticker)
    data['ticker']= ticker
    ticker_data.append(data)
ticker_data_df = pd.concat(ticker_data)
ticker_data_df = ticker_data_df.groupby(['ticker'], group_keys=False).apply(lambda x: get_return_per_interval(x, ['open', 'close', 'high', 'low']))



Getting data for CFXBUSD
Getting data for BTCBUSD


In [146]:
px.line(ticker_data_df.reset_index().sort_values('dateTime'), 'dateTime', 'high', color ='ticker')

In [169]:
px.line(ticker_data_df.reset_index(), 'dateTime', 'open_return', color='ticker')

In [226]:
open_ret_per_time = ticker_data_df.groupby([ticker_data_df.dateTime.dt.time, 'ticker'])['open_return'].mean()
px.line(open_ret_per_time.reset_index(), 'dateTime', 'open_return', color='ticker', title='average open returns at each half-hour')

In [225]:
df=ticker_data_df.set_index('dateTime')
expanding_corr = df[df.ticker=='CFXBUSD']['open'].expanding(5).corr(df[df.ticker=='BTCBUSD']['open'])
px.line(expanding_corr, title='correlation of opening prices')

In [219]:
a=ticker_data_df[ticker_data_df.ticker=='CFXBUSD']['open']
b=ticker_data_df[ticker_data_df.ticker=='BTCBUSD']['open']
a.expanding(10).corr(b)

0            NaN
1            NaN
2            NaN
3            NaN
4            NaN
          ...   
1435    0.911092
1436    0.911357
1437    0.911653
1438    0.911943
1439    0.912224
Name: open, Length: 1440, dtype: float64

In [223]:
df.shape

(2880, 11)